In [11]:
import tkinter as tk
from tkinter import ttk
from tkinter.messagebox import *

from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from matplotlib import cm


from matplotlib import pyplot as plt
from toydaq import Motor, Diode, scan_iter

from functools import partial  

from threading import Thread

from tqdm import tqdm
import numpy as np

from toydaq.tricks import MotorTricks
from functools import wraps



#add a class to the model for add_motor that takes this title, and units, and checks that there isn't a motor of the same name in the dict self.motors (so you can be sure).
#Then just index self.motors directly with that key.

 
class Model():
    # Below two class variable.
    #So it's sort of static and shared between all instances. The only thing to be careful about is don't use class variables for stuff that changes per instantiation, because if you update one instance the other classes would be changed too.
    #when you define it on the class, you don't use self. Self is in reference to the instance, which exists when you create it and call a function.
    #It's not actually protected, you can change it. But you don't want to change it.
    valid_units = ["m", "nm", "mm"]
    valid_diode_names=["INTENSITY", "COUNTER", "SIGNAL", "XRF"]
    
    def __init__(self):
        """
        Make motors and diodes available in the model with their methods and attributes.
        better in the future: load from a config file
        """
        self.motors = {}
        self.diodes = {}

    def _check_valid_units(self, units):
        """
        Ensure the user has provided a valid unit name
        """
        if units not in self.valid_units:
            options = ", ".join(self.valid_units)
            raise ValueError(f"{units} is not a known unit type. Options are {options}")

    def add_motor(self, name, units):
        """
        Add a motor to the model
        """
        if name in self.motors:
            raise ValueError(
                f"Motor names must be unique: {name} is already a defined motor."
            )

        # Allow any casing
        units = units.lower()
        self._check_valid_units(units)
        print(f"Adding motor {name} with units {units}")
        self.motors[name] = Motor(name, units)

    def _check_valid_diode_names(self, diode_name):
        """
        Ensure the user has provided a valid diode name
        """   
        if diode_name not in self.valid_diode_names:
            options = ", ".join(self.valid_diode_names)
            raise ValueError(f"{diode_name} is not a known sensor diode name. Options are {options}")

    def add_diode(self, diode_name):
        """
        Add a diode to the model
        """
        if diode_name in self.diodes:
            raise ValueError(
                f"Diode names must be unique: {diode_name} is already a defined diode."
            )
        
        self._check_valid_diode_names(diode_name)
        print(f"Adding diode {diode_name}")
        self.diodes[diode_name]=Diode(diode_name)
  
    def diode_names(self):
        return [self.diodes[label].name for label in self.diodes]

    def motor_names(self):
        return [self.motors[label].name for label in self.motors]

    
    

     #Motornamen aus dem Model 
     # Model: declare motor names als Methode def motor_name return: Motornamen sein 
     # Ich kann die Daten vom Model nur mit Methoden erreichen
     # motor_pos(motor_name)
     #   motor_pos.get , return motor_pos.get 
     # OOP Datahiding , Nachrichten   
 
class View():
    # The view will be responsible for displaying widgets
    # and getting some information from user and from model.

    def __init__(self, model):

        self._model = model

        self._master = tk.Tk()
 
        self._frame = tk.Frame(self._master)
  
        self.fig = Figure(figsize=(7.5, 4), dpi=80)

        self.ax0 = self.fig.add_subplot()
        self._frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=1)
        
        self.canvas = FigureCanvasTkAgg(self.fig, master=self._frame)
        #self.canvas.draw()

        toolbar = NavigationToolbar2Tk(self.canvas, self._frame)
        toolbar.update()
        self.canvas.get_tk_widget().pack(side=tk.LEFT, fill=tk.BOTH, expand=1)

        self.sidepanel = SidePanel(self._frame, self,  self._model)   #parent,  guicontrol, model as parame for SidePannel, this is how you establish communication :-)
        self.motorpanel= MotorPanel(self._frame, self, self._model)   #parent,  guicontrol, model as parame for Motorpanel 


        self.cbVariables={}
        self.cb={}
        #self.cbTexts={}
        #create checkboxes for diodes
        for diode_name in self._model.diode_names():
            self.cbVariables[diode_name]=tk.IntVar() #default value=0   #master=self.view.sidepanel.frame3
            #self.cbTexts[diode_name] = tk.StringVar()
            self.cb[diode_name]=ttk.Checkbutton(self.sidepanel.frame3,text=diode_name, variable=self.cbVariables[diode_name], state=['!alternate','!selected'], offvalue=0, onvalue=1, command=lambda diode_name=diode_name: self.on_cb_diode_selection(diode_name))
            self.cb[diode_name].pack(side=tk.TOP, anchor=tk.W) 
            print(f"Test on diode {diode_name} state at start: !alternate: {self.cb[diode_name].instate(['!alternate'])}, !selected: {self.cb[diode_name].instate(['!selected'])}") 
     
        
    def on_create_scanplot(self, m):
        self.sidepanel.plotBut.bind("<Button>", m)

    def on_clear(self, m):
        self.sidepanel.clearButton.bind("<Button>", m)

    def on_combobox_selection(self, m):  
        self.sidepanel.motor_selCombo.bind("<<ComboboxSelected>>", m)    

    def on_show_scan_settings(self,m):
        self.motorpanel.show_scan_settingsButton.bind("<Button>", m)

    def on_cb_diode_selection(self,diode_name): 
        """ Prints out the state changes for a diode """
        print(f"Diode {diode_name} is {self.cb[diode_name].state()}")

        
    def run(self):
        self._master.geometry("1200x600")
        self._master.title("Tkinter MVC example")
        self._master.deiconify()
        self._master.mainloop()

    def on_motor_selection(self, mot_selected):
        
        # shows the selected motor and its position in the label
        self.sidepanel.curr_motor_valLabel.configure(text=self._model.motors[mot_selected])

        print(f"{mot_selected} was selected.")

        # adjust the units shown above the motor input values, when the motor is changed by the user 
        for column in self.motorpanel.motor_inputTexts:
            self.motorpanel.motor_inputTexts[column].set(f"{column} [{self._model.motors[mot_selected].units}]")
           
        # return motor as object 
        return self._model.motors[mot_selected]

    def get_motor_selection(self):
        "get current motor selection from combobox and return motor"
        try:
            if self.sidepanel.motor_selCombo.get() == "":  
                raise ValueError('No motor selected')
            else:
                print('Current selected motor ', self.sidepanel.motor_selCombo.get())

                #returns only the motor name, i.e. str
                #return self.sidepanel.motor_selCombo.get()
                # returns a motor object
                return self._model.motors[self.sidepanel.motor_selCombo.get()]

        except ValueError as e:
            showwarning(title='Empty motor selection', message="Select motor.")
        

    def get_diode_selection(self):
        """ Returns which diodes is selected. Two choices: by state or by variable. I went with by variable."""
        #for label in self.cbVariables:
        #    print(label, self.cbVariables[label].get())

        for diode_name in self.cb:
            if self.cb[diode_name].instate(['selected']) == True:
                print(f"Diode {diode_name} is selected.")

        diodes_cb_values=[v.get() for v in self.cbVariables.values()]
        print(diodes_cb_values)

        if all([ w == 0 for w in diodes_cb_values]):
            self.no_diode_selected()
            return
        else:
            all_selected_diodes={k:v for (k,v) in self.cbVariables.items() if v.get() == 1}
            print(all_selected_diodes, "type is", type(all_selected_diodes))
        
        return all_selected_diodes

    def show_scan_settings(self): 
        """ current motor, start, stop, stepsize is shown """
        curr_mot_selected=self.get_motor_selection().name  #str
        print(f"{curr_mot_selected} is selected.")
        # access the ttk.Entry 
        out_strings=[]
        for column in self.motorpanel.motor_inputTexts:
            print(print('Identify', self.motorpanel.entries[column]))
            # Label : Value
            out_str=f"{self.motorpanel.motor_inputTexts[column].get()}: {self.motorpanel.entriesValues[column].get()}"
            print(out_str)
            out_strings.append(out_str)

            # Both options possible, get only one, I go for the Variables of the ttk.Entry
            #print(f"This is the current Variable for {self.motorpanel.entries[column]}: {self.motorpanel.entriesValues[column].get()}")
            #print("This is the current value via.get on the ttk.Entry", self.motorpanel.entries[column].get())
        showinfo("Current scan settings", f"Motor: {curr_mot_selected}, {str(out_strings)[1:-1]}")
        
    def retrieve_scan_settings(self):
        """ Collect scan settings """

        mot_selected=self.get_motor_selection()
        mot_scan_settings=self.motorpanel.entriesValues

        return (mot_selected, mot_scan_settings)
      


    def line_colors(self, number_of_lines):
        # create better line colors
        start = 0.0
        stop = 1.0
        cm_subsection = np.linspace(start, stop, number_of_lines)
        colors = [cm.jet(x) for x in cm_subsection]
        return colors
        
    def modify_1d_plot(self, mot, diodes):
        colors=self.line_colors(len(diodes))
        self.ax0.set_title('1D Scan')
        self.ax0.set_xlabel(f"{mot.name} [{mot.units}]")
    
    def plot_1d_scan(self, mot_selected, dio_selected, mot_values, dio_values):
        # some graphic modificaton
        self.modify_1d_plot(mot_selected, dio_selected)
        marker = ['o', 'v', '^', '<', '>', 's', '8', 'p']
        colors = ['b', 'g', 'r', 'c', 'm', 'k']
        handles=[]
        labels=[]
        for i in range(len(dio_selected)):
            line, =self.ax0.plot(mot_values, dio_values[i],  marker=marker[i], color=colors[i])
            #keep infos on legend.
            handles.append(line)
            labels.append(dio_selected[i])
        self.ax0.legend(handles,labels, loc="best")
        #this updates the plot (see Matplotlib doc)
        self.canvas.draw_idle()
        self.fig.canvas.flush_events()

    def no_motor_selected(self):
        showwarning(title='1D scan warning', message="No motor selected for 1D scan.")
        
    
    def no_diode_selected(self):
        showwarning(title='1D scan warning', message="No diode selected for 1D scan.")
        

class SidePanel():
    def __init__(self, root, guicontrol, model):
        self.guicontrol=guicontrol
        self._model=model
        self._frame2 = tk.Frame(root)
        self._frame2.pack(side="top", fill=tk.BOTH, expand=1)
        self.plotBut = tk.Button(self._frame2, text="Plot ")
        self.plotBut.pack(side="top", fill=tk.BOTH)
        self.clearButton = tk.Button(self._frame2, text="Clear")
        self.clearButton.pack(side="top", fill=tk.BOTH)


        # Create a frame
        self.motor_sel_title=ttk.Label(self._frame2, text='Motor selection').pack()
        motor_name=tk.StringVar() #Motornames are strings
        self.motor_selCombo=ttk.Combobox(self._frame2, textvariable=motor_name)
        self.motor_selCombo['values']=self._model.motor_names()
        self.motor_selCombo['state'] = 'readonly'
        #motor_sel.grid(column=0, row=1)
        self.motor_selCombo.pack(side="top", fill=tk.BOTH)
    

        #Create display for current motor values
        self.curr_motor_valLabel=ttk.Label(self._frame2)
        self.curr_motor_valLabel.pack(side="top", fill=tk.BOTH)

         # Create LabelFrame for diodes
        self.frame3=tk.LabelFrame(self._frame2, height=150, text="Diodes")
        self.frame3.pack(side=tk.TOP, fill=tk.X, expand=0, anchor=tk.NW)
        self.frame3.config(highlightcolor='blue', highlightthickness=5, highlightbackground='blue', labelanchor="nw")

        # we bind the motorpanel to the sidepanel and feed the model through
        #self.motorpanel= MotorPanel(self._frame2, self._model)
 
class MotorPanel():
    #You cannot use both pack and grid on widgets that have the same master.
    #SidePanel._frame2 is a child of MainWindow._master
    #MotorPanel.frame4 is a child of MainWindow._master too
    #both have same parent, both needs to use either pack or grid   
    #def __init__(self, root, model):
    def __init__(self, root, guicontrol, model):
        self.guicontrol=guicontrol
        self._model=model
        self.frame4=tk.Frame(root)
        self.frame4.pack(side='top',fill=tk.BOTH, expand=1) # or grid(). But grid() does not work here.  #Error was: forgot to pack() frame4. so its child widgets are not visible too
        
        self.entries = {}
        self.entriesValues = {}
        self.motor_input_label=["Start", "Stop", "Stepsize"]
        self.motor_input_titleLabel={}
        self.motor_inputTexts ={}

        # %P = value of the entry if the edit is allowed
        vcmd = (self.frame4.register(self.onValidate), '%P', '%s', '%S','%W')

        for i, column in enumerate(self.motor_input_label):
            print(i, column)
            self.motor_inputTexts[column] =tk.StringVar(value=column)
            #self.motor_inputTexts[column].set(f"{column}") #default labels without units
            self.motor_input_titleLabel[column] = ttk.Label(self.frame4, font=('Calibri', 13), textvariable=self.motor_inputTexts[column])
            self.motor_input_titleLabel[column].grid(row=0, column=i, sticky="nswe")
           

            self.entriesValues[column]= tk.DoubleVar()
            self.entries[column] = ttk.Entry(self.frame4, font=('Calibri', 13), textvariable=self.entriesValues[column], justify='left', validate="key", validatecommand=vcmd)
            self.entries[column].grid(row=1, column=i, sticky="nswe")

            
        #lets create a push button
        self.show_scan_settingsButton=ttk.Button(self.frame4,text="Show scan settings")
        self.show_scan_settingsButton.grid(row=2,columnspan=len(self.motor_input_label))
        
        #If I use pack() on frame4, all childs need to use pack as well, please? - NO
        #- frame4 have root as its parent -- every child of root is using pack, so frame4 can be packed
        #- entries created have frame4 as their parent -- every child of frame4 is using grid, so entry can be grid'ed
        #the manager used for parent doesnt affect the manager to be used for child
        #not related
        #remember this, if you packed a child widget, all other child widgets of that master needs to be packed
        #BUT that doesnt mean you will have to pack the child widgets of a child here

        #SidePanel._frame2 is a child of MainWindow._master
        #MotorPanel.frame4 is a child of MainWindow._master too
        #both have same parent, both needs to use either pack or grid

        #these are to give them weights, if two rows have equal weights, they will divide the space 1:1
        #the more weight, the more space they will take

    def onValidate(self, P, s, S, W):
        """This should validate the input for each ttk.Entry
        %P = value of the entry if the edit is allowed
        %s = value of entry prior to editing
        %S = the text string being inserted or deleted, if any
        %W = the tk name of the widget
        """
        if P.strip() == "":
            # allow blank string
            return True
        elif P.strip() == "-":
            return True
        elif P.strip() == "+":
            return True
        try:
            float(P)
            return True
        except ValueError:
            print(f'float({P}) raised ValueError in {W}.')
            showwarning(title=f'ValueError in {W}', message=f"{P} is not a valid input.")
            return False

        #Create 3 entry boxes for motor, start, stop, stepsize
        # Create entry limits for software limits
        # Create Go button
        
    

class Controller():
    def __init__(self, model, view):
        
        self.model = model
        self.view=view
          
        self.view.on_create_scanplot(self.create_scan_plot)
          
        self.view.on_clear(self.clear)

        self.view.on_combobox_selection(self.motor_selection)

        self.view.on_show_scan_settings(self.show_scan_settings)

            
 
    def clear(self, event):
        self.view.ax0.clear()
        self.view.canvas.draw()

    
    def prepare_1d_scan_settings(self):
        """ Retrieves relevant information for scan and prepapres scan dict for output (1D scan)"""

        mot_selected, scan_values=self.view.retrieve_scan_settings()

        #["Start", "Stop", "Stepsize"]
        start= scan_values["Start"].get()
        stop= scan_values["Stop"].get()
        stepsize= scan_values["Stepsize"].get()
       
        #check
        try:
            if stepsize > np.absolute(stop - start):
                raise ValueError("Stepszie larger than total scan range.")
            
        except ValueError as e:
            showwarning(title='Stepsize too large.', message="Stepszie larger than total scan range.")
            return

        try:
            if start > stop:
                raise ValueError("Start>Stop")
        except ValueError as e:
            showwarning(title="Start>Stop", message="Please select start < stop.")

        try:
            if stepsize < 0:
                raise ValueError("Stepsize negative.")

        except ValueError as e:
            showwarning(title="Stepsize negative." , message="Choose positive Stepsize.") 
        
        try:
            if stepsize==0:
                raise ValueError(f"Stepsize is {stepsize}")
        except ValueError as e:
            showwarning(title=f"Stepsize {stepsize}." , message="Choose different Stepsize.") 
        
        
        scan_dict={}
        scan_dict["mot_sel"]=mot_selected
        scan_dict["Start"]=start
        scan_dict["Stop"]=stop
        scan_dict["Stepsize"]=stepsize
        return scan_dict

    def create_scan_plot(self, event):
       
        #get the motor
        if not self.view.get_motor_selection():
            self.view.no_motor_selected()
            return
        else:
            mot_selected=self.view.get_motor_selection()
            print(f"Selected motor for scan is {mot_selected}.")
        
        #get the diodes
        dio_selected_dict=self.view.get_diode_selection()
        print("Now in create scan plot, print selected diodes ",  dio_selected_dict)

        # get the keys
        dio_selected=[k for k,v in  dio_selected_dict.items()]
        num_dio_sel=len(dio_selected)
        print(f"Selected diodes for scan are {dio_selected}.")
       
        # get scan configuration
        scan_dict=self.prepare_1d_scan_settings()
    

        #clear graph
        self.view.ax0.clear()
        # prepare for plot
        #self.view.modify_1d_plot(self.model.motors[mot_selected], dio_selected)
        #self.view.modify_1d_plot(mot_selected, dio_selected)
                  
        x_values=[]
        diode_values = [[] for i in range(0, num_dio_sel)]

        # Pseudo exectuition of motor
        #for pos in scan_iter(self.model.motors[mot_selected], start, stop, stepsize, cb=None, show_progress=False):
        for pos in scan_iter(scan_dict["mot_sel"], scan_dict["Start"], scan_dict["Stop"], scan_dict["Stepsize"], cb=None, show_progress=False):


            #save motor positions
            x_values.append(pos)
            # read out and save diode positions
            for i in range(num_dio_sel):
                diode_values[i].append(self.model.diodes[dio_selected[i]].get())
                #print(dio_selected[i])
                #print(self.model.diodes[dio_selected[i]])

            # this is the function that sends data for plotting to view
            self.view.plot_1d_scan(mot_selected=scan_dict["mot_sel"],dio_selected=dio_selected, mot_values=x_values, dio_values=diode_values)
     

        #TODO: 
        # move motore per step and read out at every step
        # if I had better access to the scan function, this whole function would be nicer 


    def motor_selection(self, event):
        # this method is directly bound to the function, so we can make use of this option
        mot_selected = event.widget.get()
        #show the current motor value after selection in the GUI
        self.view.on_motor_selection(mot_selected)


    def show_scan_settings(self, event):
        self.view.show_scan_settings()

  

    #def oncheck_sensors(self,event):
    #    print("you clicked:", event.widget)
    #    print('Das ist der Zustand', self.cbVariables[0].get())
    #    print('Das ist der Value', self.cb[0]["text"].get())
    #    #print("Value of checkbox",event.widget.get() )
        

        
    def go_button(self, event):
        #collect which sensors are read
        #which motor is selected
        #start scan
        pass


def main():

    # create model and add diodes and motors
    # TODO: hide adding of motors in Model?
    model = Model()
    motors = {"MY-LARGE-MOTOR": "m", "MY-TINY-MOTOR": "nm", "MY-NORMAL-MOTOR": "mm"}
    for motor_name, units in motors.items():
        model.add_motor(motor_name, units)
   
    diodes = {"INTENSITY": None, "COUNTER": None, "SIGNAL": None, "XRF": None} 
    for diode_name in diodes.keys():
        model.add_diode(diode_name) 

    # prepare GUI
    view = View(model)
   

    # inject into Controller
    c = Controller(model, view)
   
    # start the GUI
    view.run()



    diode_names = model.diode_names()
    for name in diode_names:
        print(name)

if __name__ == '__main__':
    main()
  

Adding motor MY-LARGE-MOTOR with units m
Adding motor MY-TINY-MOTOR with units nm
Adding motor MY-NORMAL-MOTOR with units mm
Adding diode INTENSITY
Adding diode COUNTER
Adding diode SIGNAL
Adding diode XRF
0 Start
1 Stop
2 Stepsize
Test on diode INTENSITY state at start: !alternate: True, !selected: True
Test on diode COUNTER state at start: !alternate: True, !selected: True
Test on diode SIGNAL state at start: !alternate: True, !selected: True
Test on diode XRF state at start: !alternate: True, !selected: True
MY-LARGE-MOTOR was selected.
Diode COUNTER is ('active', 'focus', 'selected', 'hover')
Diode COUNTER is ('active', 'focus', 'hover')
Current selected motor  MY-LARGE-MOTOR
Current selected motor  MY-LARGE-MOTOR
Selected motor for scan is Motor 'MY-LARGE-MOTOR' at 0.019083960897386475 m.
[0, 0, 0, 0]
Now in create scan plot, print selected diodes  None


Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/envs/psi/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
  File "/var/folders/zs/1bljp9y15h9bxppw44pk88gh0000gn/T/ipykernel_10967/580221645.py", line 494, in create_scan_plot
    dio_selected=[k for k,v in  dio_selected_dict.items()]
AttributeError: 'NoneType' object has no attribute 'items'


Diode COUNTER is ('active', 'focus', 'selected', 'hover')
Diode INTENSITY is ('active', 'focus', 'selected', 'hover')
Current selected motor  MY-LARGE-MOTOR
Current selected motor  MY-LARGE-MOTOR
Selected motor for scan is Motor 'MY-LARGE-MOTOR' at 0.05144462271476837 m.
Diode INTENSITY is selected.
Diode COUNTER is selected.
[1, 1, 0, 0]
{'INTENSITY': <tkinter.IntVar object at 0x114b950c0>, 'COUNTER': <tkinter.IntVar object at 0x114b96b90>} type is <class 'dict'>
Now in create scan plot, print selected diodes  {'INTENSITY': <tkinter.IntVar object at 0x114b950c0>, 'COUNTER': <tkinter.IntVar object at 0x114b96b90>}
Selected diodes for scan are ['INTENSITY', 'COUNTER'].
Current selected motor  MY-LARGE-MOTOR


Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/envs/psi/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
  File "/var/folders/zs/1bljp9y15h9bxppw44pk88gh0000gn/T/ipykernel_10967/580221645.py", line 513, in create_scan_plot
    for pos in scan_iter(scan_dict["mot_sel"], scan_dict["Start"], scan_dict["Stop"], scan_dict["Stepsize"], cb=None, show_progress=False):
  File "/Users/gudlo523/Library/CloudStorage/OneDrive-LundUniversity/PSI_codeassignment/toydaq/toydaq/tricks.py", line 72, in wrapper
    yield from func(mot, start, stop, *args, **kwargs)
  File "/Users/gudlo523/Library/CloudStorage/OneDrive-LundUniversity/PSI_codeassignment/toydaq/toydaq/scan.py", line 40, in scan_iter
    nsteps = int(round(total_range / step))
ZeroDivisionError: float division by zero


Current selected motor  MY-LARGE-MOTOR
Current selected motor  MY-LARGE-MOTOR
Selected motor for scan is Motor 'MY-LARGE-MOTOR' at 0.05171067397857758 m.
Diode INTENSITY is selected.
Diode COUNTER is selected.
[1, 1, 0, 0]
{'INTENSITY': <tkinter.IntVar object at 0x114b950c0>, 'COUNTER': <tkinter.IntVar object at 0x114b96b90>} type is <class 'dict'>
Now in create scan plot, print selected diodes  {'INTENSITY': <tkinter.IntVar object at 0x114b950c0>, 'COUNTER': <tkinter.IntVar object at 0x114b96b90>}
Selected diodes for scan are ['INTENSITY', 'COUNTER'].
Current selected motor  MY-LARGE-MOTOR


Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/envs/psi/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
  File "/var/folders/zs/1bljp9y15h9bxppw44pk88gh0000gn/T/ipykernel_10967/580221645.py", line 513, in create_scan_plot
    for pos in scan_iter(scan_dict["mot_sel"], scan_dict["Start"], scan_dict["Stop"], scan_dict["Stepsize"], cb=None, show_progress=False):
  File "/Users/gudlo523/Library/CloudStorage/OneDrive-LundUniversity/PSI_codeassignment/toydaq/toydaq/tricks.py", line 72, in wrapper
    yield from func(mot, start, stop, *args, **kwargs)
  File "/Users/gudlo523/Library/CloudStorage/OneDrive-LundUniversity/PSI_codeassignment/toydaq/toydaq/scan.py", line 40, in scan_iter
    nsteps = int(round(total_range / step))
ZeroDivisionError: float division by zero


Current selected motor  MY-LARGE-MOTOR
Current selected motor  MY-LARGE-MOTOR
Selected motor for scan is Motor 'MY-LARGE-MOTOR' at 0.06949333184103723 m.
Diode INTENSITY is selected.
Diode COUNTER is selected.
[1, 1, 0, 0]
{'INTENSITY': <tkinter.IntVar object at 0x114b950c0>, 'COUNTER': <tkinter.IntVar object at 0x114b96b90>} type is <class 'dict'>
Now in create scan plot, print selected diodes  {'INTENSITY': <tkinter.IntVar object at 0x114b950c0>, 'COUNTER': <tkinter.IntVar object at 0x114b96b90>}
Selected diodes for scan are ['INTENSITY', 'COUNTER'].
Current selected motor  MY-LARGE-MOTOR


Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/envs/psi/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
  File "/var/folders/zs/1bljp9y15h9bxppw44pk88gh0000gn/T/ipykernel_10967/580221645.py", line 513, in create_scan_plot
    for pos in scan_iter(scan_dict["mot_sel"], scan_dict["Start"], scan_dict["Stop"], scan_dict["Stepsize"], cb=None, show_progress=False):
  File "/Users/gudlo523/Library/CloudStorage/OneDrive-LundUniversity/PSI_codeassignment/toydaq/toydaq/tricks.py", line 72, in wrapper
    yield from func(mot, start, stop, *args, **kwargs)
  File "/Users/gudlo523/Library/CloudStorage/OneDrive-LundUniversity/PSI_codeassignment/toydaq/toydaq/scan.py", line 40, in scan_iter
    nsteps = int(round(total_range / step))
ZeroDivisionError: float division by zero


INTENSITY
COUNTER
SIGNAL
XRF


In [4]:
from toydaq import Motor, Diode, scan_iter

mot1 = Motor("MY-LARGE-MOTOR", units="m")
mot2 = Motor("MY-TINY-MOTOR", units="nm")
mot3 = Motor("MY-NORMAL-MOTOR", units="mm")


motor_group=[mot1,mot2, mot3]
motor_names=[mot.name for mot in motor_group]
motor_values=[mot.get() for mot in motor_group]
print(motor_names)
print(motor_values)

{mot.name: mot.get() for mot in motor_group}


dio1 = Diode("INTENSITY")
dio2 = Diode("COUNTER")
dio3 = Diode("SIGNAL")
diode_group=[dio1,dio2, dio3]

print(dio1.get())
print(dio1)
dio1.name

['MY-LARGE-MOTOR', 'MY-TINY-MOTOR', 'MY-NORMAL-MOTOR']
[0.07560159861330784, 0.023693267694386632, 0.05106162859840953]
1.0
Diode 'INTENSITY' at 1.0


'INTENSITY'

In [5]:
from toydaq import Motor, Diode, scan_iter
motors = {"mot1": Motor("MY-LARGE-MOTOR", units="m"), "mot2": Motor("MY-TINY-MOTOR", units="nm"), "mot3":  Motor("MY-NORMAL-MOTOR", units="mm") }
#keys(self._motors) sind dann die Namen der Motoren, values(self._motors) die Motoren selbst.

motoren_group=list(motors.values())
print(motoren_group)
print(list(motors.items()))

motors["mot1"].name

[motors[label].name for label in motors]

combo_string='MY-LARGE-MOTOR'

if combo_string in [motors[label].name for label in motors]:
    print('hello world')

for index, key in enumerate(motors):
    print(index, key, motors[key].name)


inverted = {v: k for k, v in motors.items()}
print(inverted)


motors["mot1"]

[Motor 'MY-LARGE-MOTOR' at 0.05150755680443393 m, Motor 'MY-TINY-MOTOR' at 0.03829890127026993 nm, Motor 'MY-NORMAL-MOTOR' at 0.020275104264625376 mm]
[('mot1', Motor 'MY-LARGE-MOTOR' at 0.024547491754729332 m), ('mot2', Motor 'MY-TINY-MOTOR' at 0.08942868695985881 nm), ('mot3', Motor 'MY-NORMAL-MOTOR' at 0.05042512651766952 mm)]
hello world
0 mot1 MY-LARGE-MOTOR
1 mot2 MY-TINY-MOTOR
2 mot3 MY-NORMAL-MOTOR
{Motor 'MY-LARGE-MOTOR' at 0.009329597262495392 m: 'mot1', Motor 'MY-TINY-MOTOR' at 0.07101241962442202 nm: 'mot2', Motor 'MY-NORMAL-MOTOR' at 0.07370657320116342 mm: 'mot3'}


Motor 'MY-LARGE-MOTOR' at 0.08831109842414825 m

In [6]:
#add a class to the model for add_motor that takes this title, and units, and checks that there isn't a motor of the same name in the dict self.motors (so you can be sure).
#Then just index self.motors directly with that key.
motor_list=[Motor("MY-LARGE-MOTOR", units="m"),Motor("MY-TINY-MOTOR", units="nm"), Motor("MY-NORMAL-MOTOR", units="mm")]
motor_dict={item.name:item for item in motor_list}
print(motor_dict)



{'MY-LARGE-MOTOR': Motor 'MY-LARGE-MOTOR' at 0.08456784095372341 m, 'MY-TINY-MOTOR': Motor 'MY-TINY-MOTOR' at 0.058711941612594486 nm, 'MY-NORMAL-MOTOR': Motor 'MY-NORMAL-MOTOR' at 0.05561134252142315 mm}


In [7]:
diodes= { "dio1": Diode("INTENSITY"), "dio2" : Diode("COUNTER") , "dio3" :Diode("SIGNAL") }
[v.get() for v in diodes.values()]



[1.0, 1.0, 1.0]

In [8]:
from toydaq import Motor, Diode, scan_iter
motors = {"mot1": Motor("MY-LARGE-MOTOR", units="m"), "mot2": Motor("MY-TINY-MOTOR", units="nm"), "mot3":  Motor("MY-NORMAL-MOTOR", units="mm") }

motors['mot1']._active
motors['mot1'].units
motors['mot1'].name
print(motors['mot1']._set_range)

#how to tackle here the value of all motors in the dict
[v.get() for k,v in motors.items()]

[v for v in motors.keys()]  #mot1
[v for v in motors.values()]  #Motor 'MY-TINY-MOTOR' at 0.033945889684259024 nm
[v.get() for v in motors.values()] #[0.0034337422024070864, 0.004942363071030331, 0.0068243804738683965]

print(Motor("MY-LARGE-MOTOR", units="m"), Diode("INTENSITY"))  #Motor 'MY-LARGE-MOTOR' at 0.09135737565200489 m Diode 'INTENSITY' at 1.0



<bound method MotorTricks._set_range of Motor 'MY-LARGE-MOTOR' at 0.05761280633436232 m>
Motor 'MY-LARGE-MOTOR' at 0.05693815045765059 m Diode 'INTENSITY' at 1.0


In [9]:
motors = {"mot1": Motor("MY-LARGE-MOTOR", units="m"), "mot2": Motor("MY-TINY-MOTOR", units="nm"), "mot3":  Motor("MY-NORMAL-MOTOR", units="mm") }
motors["mot1", 'mot2"]


SyntaxError: unterminated string literal (detected at line 2) (3808959923.py, line 2)

In [9]:
a, b, c = 0,0,0.0
len({a, b, c}) == 1
(2-2)/0

ZeroDivisionError: division by zero